In [26]:
import random

In [1]:
# ('Charlie', 10)list index out of range ('Charlie', 10)

In [25]:
class Player:
    """
    Contains three methods
        1. setup_ships: takes no parameters and returns a list of lists showing the start position
                        of showing the start positions of your ships
                        Block of row i and column j, ship label 1-5 & 0 means no ship

        2. take_turn:   returns
                        -- a list of (row, column) tuples describing your shots
                        -- eg: [(1, 1), (1, 3), (1, 5), (2, 1) (4, 5)]
                        -- Note: should be equal to the number of ships present (check function)
                        -- or a tuple that has a ship number as the first element and the direction
                        -- of its moving value
                        -- 0: up, 1: right, 2: down, 3: left  (dict)

                        TODO: Do not take more shots than the number of ships
                        TODO: Do not move the ships outside the grid or radioactive zone

        3. get_name:   returns a string that is the name of the Player

    """
    import random
    from collections import defaultdict
    
    def __init__(self):
        
        self.player_battleground = [[0 for i in range(10)] for j in range(10)]
        self.opponent_battleground = [[0 for i in range(10)] for j in range(10)]
        
        self.ships_dd = Player.defaultdict(int)
        self.status = []
        
        self.potential_list = []
        for i in range(10):
            for j in range (10):
                self.potential_list.append((i,j))
                
        # number of turns
        self.counter = 0
        

    def setup_ships(self):
        '''

        :return: a 10x10 grid with the location of our ships
        '''
        grid = [[0 for i in range(10)] for j in range(10)]
        
        # ship numbers
        ship = [i for i in range(1,6)]

        # randomly shuffle the ship list
        Player.random.shuffle(ship)
         
        for ship_number in ship:
            control = True
            
            while control:
                # randomly select a point on the grid
                ship_start_position = self.random_position()

                if self.vacant(ship_start_position, ship_number, grid):
                    for row in range(0, ship_number):
                        #print(f'{ship_start_position} ship : {ship_number}')
                        grid[ship_start_position[0] + row][ship_start_position[1]] = ship_number

                    control = False        
                
        self.player_battleground = grid
        return grid

    def take_turn(self, history):
        '''

        :param history: records the game
                        list that contains one entry per turn
                        dict with 3 keys
                            - 'shots' : list of shots made in the turn
                            - 'hits' : an integer number of hits that your shots made
                            - 'incoming' : opponents list of shots, [] if moved

        :return: a list of tuple/s describing the shots or changed location
                depending on the strategy
        '''
        last_shot = []
        if self.counter > 0:
            last_shot = history[-1]['incoming']
    
        self.player_battleground = self.update_opponent_shots(self.player_battleground, last_shot)
        
        # counting the number of ships left
        self.ships_dd = Player.defaultdict(int) #reset count everytime
        #ships_dd = Player.defaultdict(int)

        for row in self.player_battleground:
            for point in row:
                self.ships_dd[point] += point
                
        self.status = self.ship_status(self.ships_dd)
        
        # Count the number of ships remaining
        number_of_ships = len(self.status)
        
        # Take random shots
        if self.counter > 0:
            shots_list = [number for number in self.random_selection(self.potential_list, number_of_ships)]
        else:
            shots_list = [number for number in self.random_selection(self.potential_list, 5)]      
        
        
        self.counter += 1
        
        return shots_list

    def get_name(self):
        '''

        :return: string - name of the Player
        '''
        return "Syndicate_10"
    
    # function to select a point on the grid
    def random_position(self):
        return [Player.random.randint(0,9), Player.random.randint(0,9)]
    
    # function to check if the position is vacant
    def vacant(self, pos, ship, grid):
        # checking if the ship will fit in the board
        if pos[0] + ship <= 10:
            for i in range(0, ship):
                # checking if any other ship is already present
                if grid[pos[0] + i][pos[1]] != 0:
                    return False
            return True
        return False 
    
    def random_shots(self, num = 5):
        for x in range(0,num):
            yield Player.random.randint(0,9), Player.random.randint(0,9)
            
    def update_opponent_shots(self, grid, incoming_shots):
        for x, y in incoming_shots:
            grid[x][y] = 9
        return grid
    
    def ship_status(self, ships_dd):
        status = []
        for ship in ships_dd.keys():
            # as long as the default dictionary has a key, the ship is breathing
            if ship in [1,2,3,4,5]:
                status.append((ship, ships_dd[ship]/(ship**2)))
        return status
                
    def random_selection(self, potential_list, num = 0):
        if num > len(self.potential_list):
            # If the number of shots remaining are less than the actual ships
            num = len(self.potential_list)

        for x in range(0, num):
            shot = Player.random.choice(self.potential_list)
            self.potential_list.remove(shot)

            yield shot

In [27]:
p1 = Player()

In [28]:
p1.setup_ships()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 3, 0, 0, 0, 1],
 [0, 0, 0, 5, 0, 3, 0, 0, 0, 0],
 [0, 0, 0, 5, 0, 3, 0, 4, 0, 0],
 [0, 0, 0, 5, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 5, 0, 0, 0, 4, 0, 0],
 [0, 0, 0, 5, 0, 0, 0, 4, 0, 0]]

In [30]:
p1.take_turn(history)

[(6, 1), (7, 8), (9, 6), (9, 5), (1, 4)]

In [29]:
# initial history
history = [[]]

In [31]:
p1.ships_dd

defaultdict(int, {0: 0, 2: 4, 3: 9, 1: 1, 5: 25, 4: 16})

In [29]:

#Set up the empty ports. one for the player and one for the enemy.
myport     = [ [0 for i in range(10)] for j in range(10) ]
enemyport  = [ [0 for i in range(10)] for j in range(10) ]
#Set up the probability map.
# initialize a 10x10 grid with 15
# 15 shots to take in total
enemy_probmap    = [ [15 for i in range(10)] for j in range(10) ]
ship_dd    = [ (1,1), (2,1), (3,1), (4,1), (5,1) ] # p1.ships_dd

In [30]:
enemy_probmap

[[15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15]]

In [ ]:
def update_enemy_probmap(self, history): 
        # reset enemy_probmap
        # initialize a 10x10 grid with -1
        enemy_probmap = [ [-1 for i in range(10)] for j in range(10) ]
        
        total_hit = 0
        total_shot = 0
        for turn in history:
            # record the success rate of each shot in enemy_probmap
            if turn['shots']:
                # keep updating the porbability based on the shots taken
                prob = int(turn['hits'] / len(turn['shots']) * 100 + 0.5)
                for i in turn['shots']:
                    self.enemy_probmap[i[0]][i[1]] = prob
            # counting the total number of hits and shots
            total_hit += turn['hits']
            total_shot += len(turn['shots'])
            
        # update the probablity of spots that have not been shots.
        prob = int((15 - total_hit) / (100 - total_shot) * 100 + 0.5)
        for row in range(10):
            for column in range(10):
                if self.enemy_probmap[row][column] == -1:
                    self.enemy_probmap[row][column] = prob  

In [31]:
[ [-1 for i in range(10)] for j in range(10) ]

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]

In [25]:
history = [{'shots': [(4, 3), (9, 3), (9, 5), (7, 1), (8, 0)],
  'hits': 2,
  'incoming': [(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]},
 {'shots': [(4, 7), (7, 1), (3, 7), (4, 9), (9, 2)],
  'hits': 4,
  'incoming': [(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]},
 {'shots': [(4, 8), (3, 2), (2, 7), (1, 4), (6, 2)],
  'hits': 3,
  'incoming': [(2, 3), (8, 2), (6, 3), (6, 4), (8, 2)]},
 {'shots': [(0, 0), (1, 9), (6, 9), (9, 3), (0, 8)],
  'hits': 1,
  'incoming': [(4, 1), (1, 6), (6, 3), (5, 6), (1, 2)]},
 {'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]}]

In [38]:
enemy_probmap = [ [-1 for i in range(10)] for j in range(10) ]
total_hit = 0
total_shot = 0
for turn in history:
    # record the success rate of each shot in enemy_probmap
    if turn['shots']:
        prob = int(turn['hits'] / len(turn['shots']) * 100 + 0.5)
        print(prob)
        for i in turn['shots']:
            enemy_probmap[i[0]][i[1]] = prob
    total_hit += turn['hits']
    total_shot += len(turn['shots'])
    

40
80
60
20
0


In [39]:
enemy_probmap

[[20, -1, -1, -1, -1, -1, -1, -1, 20, -1],
 [-1, -1, -1, 0, 60, -1, -1, -1, -1, 20],
 [-1, -1, -1, -1, -1, -1, -1, 60, -1, -1],
 [-1, -1, 60, -1, -1, -1, -1, 80, -1, -1],
 [-1, -1, -1, 40, -1, -1, -1, 80, 60, 80],
 [-1, -1, -1, -1, 0, -1, -1, -1, -1, -1],
 [-1, -1, 60, 0, -1, -1, -1, -1, -1, 20],
 [-1, 80, -1, -1, -1, -1, -1, -1, -1, -1],
 [40, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, 0, 20, -1, 40, -1, -1, 0, -1]]

In [40]:
total_hit

10

In [41]:
total_shot

25

In [43]:
# update the probablity of spots that have not been shots.
prob = int((15 - total_hit) / (100 - total_shot) * 100 + 0.5)
# prob = number of hits remaining
for row in range(10):
    for column in range(10):
        if enemy_probmap[row][column] == -1:
            enemy_probmap[row][column] = prob  

In [42]:
int((15 - total_hit) / (100 - total_shot) * 100 + 0.5)

7

In [66]:
int((15 - total_hit) / (100 - total_shot) * 100 + 0.5)

-29

In [44]:
enemy_probmap

[[20, 7, 7, 7, 7, 7, 7, 7, 20, 7],
 [7, 7, 7, 0, 60, 7, 7, 7, 7, 20],
 [7, 7, 7, 7, 7, 7, 7, 60, 7, 7],
 [7, 7, 60, 7, 7, 7, 7, 80, 7, 7],
 [7, 7, 7, 40, 7, 7, 7, 80, 60, 80],
 [7, 7, 7, 7, 0, 7, 7, 7, 7, 7],
 [7, 7, 60, 0, 7, 7, 7, 7, 7, 20],
 [7, 80, 7, 7, 7, 7, 7, 7, 7, 7],
 [40, 7, 7, 7, 7, 7, 7, 7, 7, 7],
 [7, 7, 0, 20, 7, 40, 7, 7, 0, 7]]

In [136]:
history = [{'shots': [(4, 3), (9, 3), (9, 5), (7, 1), (8, 0)],
  'hits': 2,
  'incoming': [(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]},
 {'shots': [(4, 7), (7, 1), (3, 7), (4, 9), (9, 2)],
  'hits': 4,
  'incoming': [(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]},
 {'shots': [(4, 8), (3, 2), (2, 7), (1, 4), (6, 2)],
  'hits': 3,
  'incoming': [(2, 3), (8, 2), (6, 3), (6, 4), (8, 2)]},
 {'shots': [(0, 0), (1, 9), (6, 9), (9, 3), (0, 8)],
  'hits': 1,
  'incoming': [(4, 1), (1, 6), (6, 3), (5, 6), (1, 2)]},
 {'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]},
{'shots': [(7, 3), (5, 2), (9, 4), (6, 1), (2, 6)], 'hits': 0, 'incoming': [(9, 1), (0, 4), (8, 3), (7, 7), (0, 9)]},
{'shots': [(4, 9), (6, 6), (9, 6), (4, 8), (2, 2)], 'hits': 1, 'incoming': [(5, 0), (1, 7), (4, 1), (1, 4), (1, 1)]},
{'shots': [(9, 8), (6, 6), (4, 8), (4, 6), (7, 1)], 'hits': 0, 'incoming': [(0, 5), (7, 0), (2, 2), (8, 7), (0, 6)]},
{'shots': [(6, 3), (3, 7), (3, 2), (2, 0), (5, 4)], 'hits': 2, 'incoming': [(9, 1), (2, 7), (0, 1), (0, 1), (6, 0)]},
{'shots': [(4, 4), (6, 4), (4, 6), (6, 1), (8, 1)], 'hits': 0, 'incoming': [(2, 9), (2, 9), (6, 9), (1, 4), (4, 2)]}]

In [68]:
enemy_probmap = [ [-1 for i in range(10)] for j in range(10) ]
        
total_hit = 0
total_shot = 0
for turn in history:
    # record the success rate of each shot in enemy_probmap
    if turn['shots']:
        # keep updating the porbability based on the shots taken
        prob = int(turn['hits'] / len(turn['shots']) * 100 + 0.5)
        print(prob)
        for i in turn['shots']:
            enemy_probmap[i[0]][i[1]] = prob
    # counting the total number of hits and shots
    total_hit += turn['hits']
    total_shot += len(turn['shots'])

# update the probablity of spots that have not been shots.
prob = int((15 - total_hit) / (100 - total_shot) * 100 + 0.5)
for row in range(10):
    for column in range(10):
        if enemy_probmap[row][column] == -1:
            enemy_probmap[row][column] = prob  

40
80
60
20
0
0
20
0
40
0


In [69]:
total_hit

13

In [70]:
total_shot

50

In [71]:
enemy_probmap

[[20, 4, 4, 4, 4, 4, 4, 4, 20, 4],
 [4, 4, 4, 0, 60, 4, 4, 4, 4, 20],
 [40, 4, 20, 4, 4, 4, 0, 60, 4, 4],
 [4, 4, 40, 4, 4, 4, 4, 40, 4, 4],
 [4, 4, 4, 40, 0, 4, 0, 80, 0, 20],
 [4, 4, 0, 4, 40, 4, 4, 4, 4, 4],
 [4, 0, 60, 40, 0, 4, 0, 4, 4, 20],
 [4, 0, 4, 0, 4, 4, 4, 4, 4, 4],
 [40, 0, 4, 4, 4, 4, 4, 4, 4, 4],
 [4, 4, 0, 20, 0, 40, 20, 4, 0, 4]]

In [ ]:
    # Function that generate the list of shots        
def shot(myship_number):              
    """
    Store the "coordinates of spots that have not been shot"
    into the list called "targets" and shuffle them to add randomness
    
    myship_number = number of ships
    """
    targets = []
    for row in range(10):
        for column in range(10):
            # if 0 in enemy port = 0 haven't made the shot; 9 if shot is made
            # enemyport is a grid of 0 and 9 (previous shots)
            if enemyport[row][column] == 0:
                targets.append((row,column))
    random.shuffle(targets)  
    """
    Evaluate likelihood score for points in the "target" list
    In this code, likelihood score (LHS) defined as
    LHS = 2*P(point) + 0.9*P(one_neighbour) + 0.8*P(2setp_neighbours)
        where P = probability of presense of a shp
    Where nibours are not available, P(point) replace P(neibours)
    Note that LHS is not probability but probability utility.
    """        
    scores =[]
    for i in range(len(targets)):   # i represent target number
        # Calculate LHS of target point i
        score = 0
        for row_step in range(-2,3):
            #  Where neighbours are not available, P(point) replace P(neibours)
            if targets[i][0] + row_step in [0,1,2,3,4,5,6,7,8,9]:
                prob = self.enemy_probmap[targets[i][0]+row_step][targets[i][1]]
            else:
                prob = self.enemy_probmap[targets[i][0]][targets[i][1]]
            score += (1-0.1*abs(row_step)) * prob

        for col_step in range(-2,3):
            #  Where neighbours are not available, P(point) replace P(neibours)
            if targets[i][1] + col_step in [0,1,2,3,4,5,6,7,8,9]:
                prob = self.enemy_probmap[targets[i][0]][targets[i][1]+col_step]
            else:
                prob = self.enemy_probmap[targets[i][0]][targets[i][1]]
            score += (1-0.1*abs(row_step)) * prob
    # Store LHS and target point i and sort at the end
        scores.append((score,i)) 
    scores.sort(reverse=True)

    # Return the top LSH points as list of tuple
    # The number of returning points is the same as the number of our ship
    shot_list =[]
    for j in range(myship_number):
        shot_row = targets[ scores[j][1] ][0]
        shot_col = targets[ scores[j][1] ][1]
        shot_list.append((shot_row,shot_col))
    return shot_list

In [53]:
targets = []
for row in range(10):
    for column in range(10):
        if enemyport[row][column] == 0:
            targets.append((row,column))

In [55]:
random.shuffle(targets)

In [57]:
for row_step in range(-2,3):
    print(row_step)

-2
-1
0
1
2


In [59]:
enemy_probmap

[[20, -29, -29, -29, -29, -29, -29, -29, 20, -29],
 [-29, -29, -29, 0, 60, -29, -29, -29, -29, 20],
 [100, -29, 80, -29, -29, -29, 60, 60, -29, -29],
 [-29, -29, 100, -29, -29, -29, -29, 100, -29, -29],
 [-29, -29, -29, 40, 60, -29, 60, 80, 100, 80],
 [-29, -29, 60, -29, 100, -29, -29, -29, -29, -29],
 [-29, 60, 60, 100, 60, -29, 100, -29, -29, 20],
 [-29, 100, -29, 60, -29, -29, -29, -29, -29, -29],
 [40, 60, -29, -29, -29, -29, -29, -29, -29, -29],
 [-29, -29, 0, 20, 60, 40, 80, -29, 100, -29]]

In [60]:
scores =[]
for i in range(len(targets)):   # i represent target number
    # Calculate LHS of target point i
    score = 0
    for row_step in range(-2,3):
        #  Where neighbours are not available, P(point) replace P(neibours)
        if targets[i][0] + row_step in [0,1,2,3,4,5,6,7,8,9]:
            prob = enemy_probmap[targets[i][0] + row_step][targets[i][1]]
        else:
            prob = enemy_probmap[targets[i][0]][targets[i][1]]
        score += (1-0.1*abs(row_step)) * prob

    for col_step in range(-2,3):
        #  Where neighbours are not available, P(point) replace P(neibours)
        if targets[i][1] + col_step in [0,1,2,3,4,5,6,7,8,9]:
            prob = enemy_probmap[targets[i][0]][targets[i][1]+col_step]
        else:
            prob = enemy_probmap[targets[i][0]][targets[i][1]]
        score += (1-0.1*abs(row_step)) * prob
        print(score)
# Store LHS and target point i and sort at the end
    scores.append((score,i))

9.399999999999991
-13.800000000000011
-37.000000000000014
42.999999999999986
19.799999999999983
125.49999999999999
157.5
205.5
182.3
230.3
-70.70000000000002
-93.90000000000002
-117.10000000000002
-140.3
-163.5
62.8
110.8
87.6
64.39999999999999
41.19999999999999
-62.60000000000001
-85.80000000000001
-69.80000000000001
-53.80000000000001
-37.80000000000001
84.6
164.6
228.6
292.6
356.6
-88.70000000000002
-111.90000000000002
-135.10000000000002
-158.3
-181.5
-150.8
-70.80000000000001
-94.00000000000001
-30.000000000000014
-53.20000000000002
-79.60000000000001
-102.80000000000001
-126.00000000000001
-110.00000000000001
-133.20000000000002
87.69999999999999
64.49999999999999
41.29999999999998
18.09999999999998
66.09999999999998
196.7
173.5
221.5
198.3
175.10000000000002
-79.60000000000001
-102.80000000000001
-126.00000000000001
-149.20000000000002
-133.20000000000002
-150.8
-174.0
-197.2
-220.39999999999998
-140.39999999999998
50.499999999999986
27.299999999999983
4.09999999999998
4.0999999

In [58]:
len(targets)

100

In [72]:
scores.sort(reverse=True)

In [73]:
scores

[(401.5, 61),
 (398.3, 89),
 (381.5, 49),
 (367.5, 73),
 (356.6, 5),
 (352.6, 41),
 (349.5, 84),
 (337.5, 63),
 (337.4, 51),
 (323.20000000000005, 64),
 (321.40000000000003, 71),
 (301.5, 60),
 (272.7, 91),
 (242.29999999999998, 33),
 (237.40000000000003, 18),
 (230.3, 1),
 (220.00000000000006, 90),
 (203.09999999999997, 72),
 (195.5, 29),
 (191.10000000000002, 37),
 (180.2, 47),
 (175.10000000000002, 10),
 (171.90000000000003, 43),
 (166.80000000000007, 46),
 (157.90000000000003, 74),
 (148.7, 80),
 (131.0, 82),
 (117.39999999999999, 24),
 (114.10000000000001, 57),
 (109.49999999999999, 40),
 (105.3, 23),
 (82.1, 44),
 (66.09999999999998, 9),
 (58.999999999999986, 96),
 (55.79999999999998, 85),
 (55.0, 42),
 (54.999999999999986, 50),
 (52.09999999999998, 13),
 (41.19999999999999, 56),
 (41.19999999999999, 3),
 (33.999999999999986, 67),
 (19.799999999999983, 0),
 (10.999999999999993, 53),
 (10.899999999999977, 95),
 (10.199999999999989, 98),
 (2.0999999999999943, 87),
 (1.9999999999999

In [63]:
myship_number = 0 
for i in p1.ships_dd.keys():
    if i[1] != 0:
        myship_number += 1

TypeError: 'int' object is not subscriptable

In [75]:
shot_list =[]
for j in range(5):
    shot_row = targets[ scores[j][1] ][0]
    shot_col = targets[ scores[j][1] ][1]
    shot_list.append((shot_row,shot_col))
    print(shot_list)

[(4, 7)]
[(4, 7), (9, 8)]
[(4, 7), (9, 8), (6, 4)]
[(4, 7), (9, 8), (6, 4), (9, 6)]
[(4, 7), (9, 8), (6, 4), (9, 6), (4, 9)]


In [1]:
test = [[]]

In [5]:
len(test)

1

In [7]:
not test

False

In [8]:
histories = [[], []]

In [9]:
histories[0]

[]

In [5]:
import random

In [16]:
def initial_shots(number_of_ships):
    placement_1 = [(1,1), (3,1), (2,2), (1,3), (3,3)]
    placement_2 = [(6,1), (8,1), (7,2), (6,3), (8,3)]
    placement_3 = [(1,6), (1,8), (2,7), (3,6), (3,8)]
    placement_4 = [(6,6), (8,6), (7,7), (8,6), (8,8)]

    shots = random.choice([placement_1, placement_2, placement_3, placement_4])
    shot_list = random.sample(shots, k=number_of_ships)#[:number_of_ships]
    
    return shot_list

In [19]:
initial_shots(1)

[(1, 8)]

In [137]:
enemy_dict = {(i, j): 15/100 for i in range(10) for j in range(10)}

In [23]:
enemy_dict

{(0, 0): 0.15,
 (0, 1): 0.15,
 (0, 2): 0.15,
 (0, 3): 0.15,
 (0, 4): 0.15,
 (0, 5): 0.15,
 (0, 6): 0.15,
 (0, 7): 0.15,
 (0, 8): 0.15,
 (0, 9): 0.15,
 (1, 0): 0.15,
 (1, 1): 0.15,
 (1, 2): 0.15,
 (1, 3): 0.15,
 (1, 4): 0.15,
 (1, 5): 0.15,
 (1, 6): 0.15,
 (1, 7): 0.15,
 (1, 8): 0.15,
 (1, 9): 0.15,
 (2, 0): 0.15,
 (2, 1): 0.15,
 (2, 2): 0.15,
 (2, 3): 0.15,
 (2, 4): 0.15,
 (2, 5): 0.15,
 (2, 6): 0.15,
 (2, 7): 0.15,
 (2, 8): 0.15,
 (2, 9): 0.15,
 (3, 0): 0.15,
 (3, 1): 0.15,
 (3, 2): 0.15,
 (3, 3): 0.15,
 (3, 4): 0.15,
 (3, 5): 0.15,
 (3, 6): 0.15,
 (3, 7): 0.15,
 (3, 8): 0.15,
 (3, 9): 0.15,
 (4, 0): 0.15,
 (4, 1): 0.15,
 (4, 2): 0.15,
 (4, 3): 0.15,
 (4, 4): 0.15,
 (4, 5): 0.15,
 (4, 6): 0.15,
 (4, 7): 0.15,
 (4, 8): 0.15,
 (4, 9): 0.15,
 (5, 0): 0.15,
 (5, 1): 0.15,
 (5, 2): 0.15,
 (5, 3): 0.15,
 (5, 4): 0.15,
 (5, 5): 0.15,
 (5, 6): 0.15,
 (5, 7): 0.15,
 (5, 8): 0.15,
 (5, 9): 0.15,
 (6, 0): 0.15,
 (6, 1): 0.15,
 (6, 2): 0.15,
 (6, 3): 0.15,
 (6, 4): 0.15,
 (6, 5): 0.15,
 (6, 6): 0

[{'shots': [(4, 3), (9, 3), (9, 5), (7, 1), (8, 0)],
  'hits': 2,
  'incoming': [(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]},
 {'shots': [(4, 7), (7, 1), (3, 7), (4, 9), (9, 2)],
  'hits': 4,
  'incoming': [(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]},
 {'shots': [(4, 8), (3, 2), (2, 7), (1, 4), (6, 2)],
  'hits': 3,
  'incoming': [(2, 3), (8, 2), (6, 3), (6, 4), (8, 2)]},
 {'shots': [(0, 0), (1, 9), (6, 9), (9, 3), (0, 8)],
  'hits': 1,
  'incoming': [(4, 1), (1, 6), (6, 3), (5, 6), (1, 2)]},
 {'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]}]

In [28]:
history[-1]['shots']

[(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)]

In [31]:
shots_taken = 0
for i in range(len(history)):
    shots_taken += len(history[i]['shots'])

In [32]:
shots_taken

25

In [33]:
from scipy import stats as st

In [38]:
st.bayes_mvs([0,3,4])[0]

Mean(statistic=2.3333333333333335, minmax=(-1.1760525778624316, 5.842719244529096))

In [39]:
# Bayes mvs statistic

In [52]:
st.energy_distance([1,1], [0,1], u_weights=None, v_weights=None)

0.7071067811865476

In [55]:
st.

TypeError: 'random_correlation_gen' object is not callable

In [77]:
enemy_dict = {(i, j): 0.15 for i in range(10) for j in range(10)}

In [64]:
def dict_mapper(enemy_dict):
    grid = [[0 for i in range(10)] for j in range(10)]
    for x, y in enemy_dict.keys():
        grid[x][y] = enemy_dict[(x,y)]
    return grid
        

In [65]:
dict_mapper(enemy_dict)

[[0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15]]

In [78]:
def probsweeper(enemy_dict, player_shots, hits, history):

    # up, down, right, left, upright, downright, up-left, down-left
    # minesweeper = [(-1, 0), (1, 0), (0, 1), (0, -1), (-1, 1), (1, 1), (-1, -1), (1, -1)]
    # up, down, right, left
    minesweeper = [(-1, 0), (1, 0), (0, 1), (0, -1)]

    hits_remaining = 15 - hits
    shots_taken = 0
    for i in range(len(history)):
        shots_taken += len(history[i]['shots'])

    # updating the enemy dictionary
    # only if it has hit!
    for i, j in player_shots:
        # opponent_battleground[i][j] = -1
        for x, y in minesweeper:
            # checking if the next position is in the board
            if (x + i) >= 0 and (x + i) <= 9 and (y + j) >= 0 and (y + j) <= 9:
                # checking if it has already not been hit
                if enemy_dict[(i + x, j + y)] != 1 or enemy_dict[(i + x, j + y)] != 0:
                    enemy_dict[(i + x, j + y)] += round((hits_remaining)/(100 - shots_taken),2)

    return enemy_dict

In [79]:
probsweeper(enemy_dict, history[-1]['shots'], 6, history)

{(0, 0): 0.15,
 (0, 1): 0.15,
 (0, 2): 0.15,
 (0, 3): 0.28,
 (0, 4): 0.15,
 (0, 5): 0.15,
 (0, 6): 0.15,
 (0, 7): 0.15,
 (0, 8): 0.15,
 (0, 9): 0.15,
 (1, 0): 0.15,
 (1, 1): 0.15,
 (1, 2): 0.28,
 (1, 3): 0.15,
 (1, 4): 0.28,
 (1, 5): 0.15,
 (1, 6): 0.15,
 (1, 7): 0.15,
 (1, 8): 0.15,
 (1, 9): 0.15,
 (2, 0): 0.15,
 (2, 1): 0.15,
 (2, 2): 0.15,
 (2, 3): 0.28,
 (2, 4): 0.15,
 (2, 5): 0.15,
 (2, 6): 0.15,
 (2, 7): 0.15,
 (2, 8): 0.15,
 (2, 9): 0.15,
 (3, 0): 0.15,
 (3, 1): 0.15,
 (3, 2): 0.15,
 (3, 3): 0.15,
 (3, 4): 0.15,
 (3, 5): 0.15,
 (3, 6): 0.15,
 (3, 7): 0.15,
 (3, 8): 0.15,
 (3, 9): 0.15,
 (4, 0): 0.15,
 (4, 1): 0.15,
 (4, 2): 0.15,
 (4, 3): 0.15,
 (4, 4): 0.28,
 (4, 5): 0.15,
 (4, 6): 0.15,
 (4, 7): 0.15,
 (4, 8): 0.15,
 (4, 9): 0.15,
 (5, 0): 0.15,
 (5, 1): 0.15,
 (5, 2): 0.15,
 (5, 3): 0.41000000000000003,
 (5, 4): 0.15,
 (5, 5): 0.28,
 (5, 6): 0.15,
 (5, 7): 0.15,
 (5, 8): 0.15,
 (5, 9): 0.15,
 (6, 0): 0.15,
 (6, 1): 0.15,
 (6, 2): 0.28,
 (6, 3): 0.15,
 (6, 4): 0.41000000000000

In [70]:
history

[{'shots': [(4, 3), (9, 3), (9, 5), (7, 1), (8, 0)],
  'hits': 2,
  'incoming': [(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]},
 {'shots': [(4, 7), (7, 1), (3, 7), (4, 9), (9, 2)],
  'hits': 4,
  'incoming': [(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]},
 {'shots': [(4, 8), (3, 2), (2, 7), (1, 4), (6, 2)],
  'hits': 3,
  'incoming': [(2, 3), (8, 2), (6, 3), (6, 4), (8, 2)]},
 {'shots': [(0, 0), (1, 9), (6, 9), (9, 3), (0, 8)],
  'hits': 1,
  'incoming': [(4, 1), (1, 6), (6, 3), (5, 6), (1, 2)]},
 {'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]},
 {'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 2,
  'incoming': [(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]}]

In [84]:
def update_enemy_dict(player_shots=history[-1]['shots'], last_hit=history[-1]['hits']):
    if player_shots:
        for i, j in player_shots:
            if last_hit > 0:
                enemy_dict[(i, j)] = 1
            else:
                enemy_dict[(i, j)] = 0

In [85]:
update_enemy_dict()

In [86]:
dict_mapper(enemy_dict)

[[0.15, 0.15, 0.15, 0.28, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.28, 1, 0.28, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.28, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.15, 0.28, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.41000000000000003, 1, 0.28, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.28, 1, 0.41000000000000003, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.15, 0.28, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15],
 [0.15, 0.15, 0.28, 0.15, 0.15, 0.15, 0.15, 0.15, 0.28, 0.15],
 [0.15, 0.28, 1, 0.28, 0.15, 0.15, 0.15, 0.28, 1, 0.28]]

In [87]:
prob_matrix = dict_mapper(enemy_dict)

In [130]:
import numpy as np

In [135]:
st.rankdata([0, 2, 3, 2], method='dense')

array([1, 2, 3, 2])

In [131]:
f = st.norm.pdf
v_bound = np.sqrt(f(np.sqrt(2))) * np.sqrt(2)
umax, vmin, vmax = np.sqrt(f(0)), -v_bound, v_bound
np.random.seed(12345)
rvs = st.rvs_ratio_uniforms(f, umax, vmin, vmax, size=2500)

AttributeError: module 'scipy.stats' has no attribute 'rvs_ratio_uniforms'

In [ ]:
def update_probabilities(prob_matrix):
    for x, y in prob_matrix:
        i

In [95]:
for (x,y), v in enemy_dict.items():
    
    minesweeper = [(-1, 0), (1, 0), (0, 1), (0, -1)]
    if (x + i) >= 0 and (x + i) <= 9 and (y + j) >= 0 and (y + j) <= 9:
        # checking if it has already not been hit
        if enemy_dict[(i + x, j + y)] != 1 or enemy_dict[(i + x, j + y)] != 0:
            enemy_dict[(i + x, j + y)] += round((hits_remaining)/(100 - shots_taken),2) 
    

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9


In [99]:
enemy_dict[(1,4)]

0.28

In [101]:
average_list = lambda x,y:[(x-1,y), (x+1, y), (x,y-1), (x,y+1)]

In [103]:
average_list(1,4)

[(0, 4), (2, 4), (1, 3), (1, 5)]

In [106]:
mysum = 0
for i in average_list(1,4):
    mysum += enemy_dict[i]
    print(enemy_dict[i])

0.15
0.15
1
0.15


In [108]:
mysum/4

0.3625

In [ ]:
def update_probabilities(prob_matrix):

In [117]:


for x,y in enemy_dict.keys():
    if (x - 1) >= 0 and (x + 1) <= 9 and (y - 1) >= 0 and (y + 1) <= 9:
        mysum = 0
        for i in average_list(x,y):
            mysum += enemy_dict[i]
            print(mysum/4)
        if enemy_dict[(x,y)] != 0 and enemy_dict[(x,y)] != 1:
            enemy_dict[(x,y)] = mysum/4

0.5090161463200407
1.1118462485112341
1.6208623948312748
2.1117767195238875
0.5090161463200407
1.0755563391565262
1.603500519037498
1.853500519037498
0.5090161463200407
1.0069436022671012
1.4703187320264757
1.9634985481260885
0.5090161463200407
1.0553528822398444
1.3053528822398444
1.8669677827515136
0.5090161463200407
1.0876658834753818
1.5544078291632601
2.1358027085303717
0.5090161463200407
1.1021324079710682
1.6360830851036612
2.2232810549116504
0.5090161463200407
1.1077751555051987
1.6635954192331113
2.2524586855830737
0.5090161463200407
1.109746002474222
1.6728606738699905
2.2617239402199534
0.5279441798809719
1.133130606002708
1.7219938723526707
2.288534065189156
0.4633751297593745
1.0549122862554112
1.6270458025527001
2.124973258499761
0.25
0.8197210015781295
1.3509643162030698
1.8973010521228737
0.4667419456878784
1.0223015999765468
1.4966268630072652
2.0752766001626064
0.5339506771325929
1.1113101580777598
1.6301293081184114
2.223245569769439
0.5558202637279126
1.146922630621

In [143]:
def update_enemy_dict(player_shots = history[-1]['shots'], hits = history[-1]['hits'], history = history):

    hits_remaining = 15 - hits
    shots_taken = 0
    for i in range(len(history)):
        shots_taken += len(history[i]['shots'])
        
    last_hit = history[-1]['hits']

    if player_shots:
        for i, j in player_shots:
            if last_hit > 0:
                enemy_dict[(i, j)] = 1
            else:
                enemy_dict[(i, j)] = 0

            for x, y in enemy_dict.keys():
                if x != i and y != j and enemy_dict[(x, y)] not in [1, 0]:
                    enemy_dict[(x, y)] += hits_remaining / (100 - shots_taken)

In [154]:
update_enemy_dict()

In [155]:
dict_mapper(enemy_dict)

[[3.013636363636363,
  1.8681818181818182,
  3.013636363636363,
  3.013636363636363,
  1.8681818181818182,
  3.013636363636363,
  2.4409090909090905,
  3.013636363636363,
  3.013636363636363,
  3.013636363636363],
 [3.013636363636363,
  1.8681818181818182,
  3.013636363636363,
  3.013636363636363,
  1.8681818181818182,
  3.013636363636363,
  2.4409090909090905,
  3.013636363636363,
  3.013636363636363,
  3.013636363636363],
 [3.013636363636363,
  1.8681818181818182,
  3.013636363636363,
  3.013636363636363,
  1.8681818181818182,
  3.013636363636363,
  2.4409090909090905,
  3.013636363636363,
  3.013636363636363,
  3.013636363636363],
 [3.013636363636363,
  1.8681818181818182,
  3.013636363636363,
  3.013636363636363,
  1.8681818181818182,
  3.013636363636363,
  2.4409090909090905,
  3.013636363636363,
  3.013636363636363,
  3.013636363636363],
 [1.8681818181818182,
  0.7227272727272727,
  1.8681818181818182,
  1.8681818181818182,
  1,
  1.8681818181818182,
  1,
  1.8681818181818182,
  

In [156]:
sorted_shot_list = sorted([(i[1], i[0]) for i in enemy_dict.items()], reverse=True)

In [157]:
sorted_shot_list

[(3.013636363636363, (9, 9)),
 (3.013636363636363, (9, 8)),
 (3.013636363636363, (9, 7)),
 (3.013636363636363, (9, 5)),
 (3.013636363636363, (9, 3)),
 (3.013636363636363, (9, 2)),
 (3.013636363636363, (9, 0)),
 (3.013636363636363, (7, 9)),
 (3.013636363636363, (7, 8)),
 (3.013636363636363, (7, 7)),
 (3.013636363636363, (7, 5)),
 (3.013636363636363, (7, 3)),
 (3.013636363636363, (7, 2)),
 (3.013636363636363, (7, 0)),
 (3.013636363636363, (5, 9)),
 (3.013636363636363, (5, 8)),
 (3.013636363636363, (5, 7)),
 (3.013636363636363, (5, 5)),
 (3.013636363636363, (5, 3)),
 (3.013636363636363, (5, 2)),
 (3.013636363636363, (5, 0)),
 (3.013636363636363, (3, 9)),
 (3.013636363636363, (3, 8)),
 (3.013636363636363, (3, 7)),
 (3.013636363636363, (3, 5)),
 (3.013636363636363, (3, 3)),
 (3.013636363636363, (3, 2)),
 (3.013636363636363, (3, 0)),
 (3.013636363636363, (2, 9)),
 (3.013636363636363, (2, 8)),
 (3.013636363636363, (2, 7)),
 (3.013636363636363, (2, 5)),
 (3.013636363636363, (2, 3)),
 (3.013636

In [153]:
history

[{'shots': [(4, 3), (9, 3), (9, 5), (7, 1), (8, 0)],
  'hits': 2,
  'incoming': [(7, 2), (2, 0), (3, 4), (9, 1), (5, 9)]},
 {'shots': [(4, 7), (7, 1), (3, 7), (4, 9), (9, 2)],
  'hits': 4,
  'incoming': [(8, 0), (0, 0), (6, 5), (1, 1), (3, 5)]},
 {'shots': [(4, 8), (3, 2), (2, 7), (1, 4), (6, 2)],
  'hits': 3,
  'incoming': [(2, 3), (8, 2), (6, 3), (6, 4), (8, 2)]},
 {'shots': [(0, 0), (1, 9), (6, 9), (9, 3), (0, 8)],
  'hits': 1,
  'incoming': [(4, 1), (1, 6), (6, 3), (5, 6), (1, 2)]},
 {'shots': [(1, 3), (6, 3), (5, 4), (9, 2), (9, 8)],
  'hits': 0,
  'incoming': [(8, 9), (3, 4), (2, 2), (6, 9), (2, 9)]},
 {'shots': [(7, 3), (5, 2), (9, 4), (6, 1), (2, 6)],
  'hits': 0,
  'incoming': [(9, 1), (0, 4), (8, 3), (7, 7), (0, 9)]},
 {'shots': [(4, 9), (6, 6), (9, 6), (4, 8), (2, 2)],
  'hits': 1,
  'incoming': [(5, 0), (1, 7), (4, 1), (1, 4), (1, 1)]},
 {'shots': [(9, 8), (6, 6), (4, 8), (4, 6), (7, 1)],
  'hits': 0,
  'incoming': [(0, 5), (7, 0), (2, 2), (8, 7), (0, 6)]},
 {'shots': [(6, 

In [21]:
enemy_probmap = [[15 for i in range(10)] for j in range(10)]

In [22]:
enemy_probmap

[[15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15],
 [15, 15, 15, 15, 15, 15, 15, 15, 15, 15]]

In [3]:
enemy_probmap[0]

[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

In [23]:
for i in range(10):
    for j in range(10):
        # lowest probability area
        if (i,j) in [(0,0), (9,9), (9,0), (0,9)]:
            enemy_probmap[i][j] = 15
        elif (i,j) in [(1,0), (0,1), (8,0), (1,9), (0,8), (9,1), (8,9), (9,8)]:
            enemy_probmap[i][j] = 16.5
        elif (i,j) in [(1,1), (1,8), (8,1), (8,8)]:
            enemy_probmap[i][j] = 18
        elif (i in [0, 9] and j >= 2 and j <= 7) or (j in [0, 9] and i >= 2 and i <= 7):
            enemy_probmap[i][j] = 18
        elif (i in [1, 8] and j >= 2 and j <= 7) or (j in [1, 8] and i >= 2 and i <= 7):
            enemy_probmap[i][j] = 19.5
        else:
            enemy_probmap[i][j] = 21

In [24]:
enemy_probmap

[[15, 16.5, 18, 18, 18, 18, 18, 18, 16.5, 15],
 [16.5, 18, 19.5, 19.5, 19.5, 19.5, 19.5, 19.5, 18, 16.5],
 [18, 19.5, 21, 21, 21, 21, 21, 21, 19.5, 18],
 [18, 19.5, 21, 21, 21, 21, 21, 21, 19.5, 18],
 [18, 19.5, 21, 21, 21, 21, 21, 21, 19.5, 18],
 [18, 19.5, 21, 21, 21, 21, 21, 21, 19.5, 18],
 [18, 19.5, 21, 21, 21, 21, 21, 21, 19.5, 18],
 [18, 19.5, 21, 21, 21, 21, 21, 21, 19.5, 18],
 [16.5, 18, 19.5, 19.5, 19.5, 19.5, 19.5, 19.5, 18, 16.5],
 [15, 16.5, 18, 18, 18, 18, 18, 18, 16.5, 15]]

In [32]:
p1.status

[(2, 1.0), (3, 1.0), (1, 1.0), (5, 1.0), (4, 1.0)]

In [37]:
8 in p1.ships_dd.keys()

False

In [41]:
pre = (1, 0)

In [42]:
type([])

list

In [43]:
not (1,0)

False

In [45]:
len((1,0))

2